In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns         

from sklearn.preprocessing import LabelEncoder
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# # from catboost import CatBoostClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from lightgbm import LGBMClassifier
# from xgboost import XGBClassifier
# from xgboost import XGBRFClassifier

# from sklearn.model_selection import train_test_split

# from sklearn.metrics import accuracy_score,recall_score
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score


In [2]:
history_df = pd.DataFrame()
file_path = 'history.csv'
if os.path.isfile(file_path) and os.path.getsize(file_path) > 0:
    history_df = pd.read_csv(file_path)
else:
    print("Error: The file is empty or does not exist.")

Error: The file is empty or does not exist.


In [3]:
'''
@param df: A pandas DataFrame
@param cols: List of columns existing in df
@return: df with cols dropped
'''
def drop_cols(df, cols):
    df = df.drop(cols, axis=1)
    return df

In [4]:
'''
@param df: A pandas DataFrame
@param cols: List of boolean columns in df
@return: df with the values in cols mapped to 0 (for False or blank) or 1 (for True)
'''
def boolean_map(df, cols):
    for col in cols:
        df[col] = df[col].fillna('f').map({'t': 1, 'f': 0})
    return df

In [5]:
'''
@param df: A pandas DataFrame
@param cols: List of datetime columns in df
@return: df with the values in cols standardized to a number between 0 and 1.
'''
def parse_time(df, cols):
    for col in cols:
        df[col] = pd.to_datetime(df[col])
        min_date = df[col].min()
        df[col] = df[col].fillna(min_date)

        df[col] = df[col].astype(int)
        min = df[col].min()
        max = df[col].max()
        df[col] = (df[col] - min) / (max - min)

    return df

In [6]:
'''
@param df: A pandas DataFrame
@param cols: List of object columns in df
@return: df with the values in cols mapped to arbitrary numbers
'''
def encode_labels(df, cols):
    label_encoder = LabelEncoder()
    for col in cols:
        df[col] = label_encoder.fit_transform(df[col])

    return df


### 1.0 Drop unused columns

We do not want our model to be trained on data from any of these columns. The code looks a bit messy because we did this step in different ways.

In [7]:
df_nathan = history_df[['id', 'type_code', 'class_id', 'inception_date', 'branch', 'include_client_consolidation', 'use_client_address', 'credit_limit_type', 'is_arp_locked', 'dividend_confirm_code']]

df_cloris = history_df[['options_trading_type', 'interest_dividend_conversion_type', 'guarantee_gtor_type', 'share_name_address_to_issuer', 'shareholder_instructions_received', 'is_tms_eligible', 'is_agent_bbs_participant', 'is_spousal_transfer', 'terminal_code', 'deceased_fair_market_value', 'discretionary_trading_authorized']]

df_cherry = history_df.iloc[:, 53:78]
df_cherry = drop_cols(df_cherry, ['esir_number', 'dup_trip_quad_code', 'function_code', 'portfolio_cost_method', 'interactive_portfolio_code'])

df_matthew = history_df.iloc[:, -27:]
df_matthew = drop_cols(df_matthew, ['is_broker_account', 'is_gl_account', 'is_control_account', 'is_plan_grandfathered'])

training_df = pd.concat([df_nathan, df_cloris, df_cherry, df_matthew], axis=1)
training_df.head(3)

KeyError: "None of [Index(['id', 'type_code', 'class_id', 'inception_date', 'branch',\n       'include_client_consolidation', 'use_client_address',\n       'credit_limit_type', 'is_arp_locked', 'dividend_confirm_code'],\n      dtype='object')] are in the [columns]"

### 1.1 Map Boolean Columns

In [ ]:
bool_nathan = ['include_client_consolidation', 'use_client_address', 'is_arp_locked']
bool_cloris = ['share_name_address_to_issuer', 'shareholder_instructions_received', 'is_tms_eligible', 'is_agent_bbs_participant', 'is_spousal_transfer', 'discretionary_trading_authorized']
bool_cherry = ['receive_general_mailings', 'has_discrete_auth', 'is_non_objecting_beneficial_owner', 'is_objecting_to_disclose_info', 'consent_to_email_delivery', 'consent_to_pay_for_mail', 'has_received_instruction']
bool_matthew = ['is_inventory_account', 'is_extract_eligible', 'is_pledged', 'is_resp', 'is_family_resp', 'is_hrdc_resp'] + ['is_olob', 'visible_in_reports', 'use_original_date_for_payment_calc']

training_df = boolean_map(training_df, bool_nathan + bool_cloris + bool_cherry + bool_matthew)

### 1.2 Parse Datetime Columns

In [ ]:
time_nathan = ['inception_date']
time_cherry = ['last_update_date']
time_matthew = ['last_maintenance_time', 'plan_effective_date', 'plan_end_date', 'rrif_original_date', 'inserted_at', 'updated_at', 'retail_last_maintenance_time']

training_df = parse_time(training_df, time_nathan + time_cherry + time_matthew)

### 1.3 Numerize Type Columns

In [ ]:
type_nathan = ['type_code', 'branch']
type_matthew = ['last_maintenance_user', 'retail_last_maintenance_user']
type_cherry = ['iso_funds_code']
training_df = encode_labels(training_df, type_nathan + type_matthew)

### 1.4 Other

In [ ]:
# Nathan
training_df.class_id.fillna(-999, inplace=True)
training_df.credit_limit_type.fillna(-999, inplace=True)
training_df.dividend_confirm_code.fillna(-999, inplace=True)

# Cloris
training_df.deceased_fair_market_value.fillna(0, inplace=True)

# Matthew
training_df.number_of_beneficiaries.fillna(0, inplace=True)
training_df.label = training_df.label.map({'No Churn': 0, 'Churn': 1})

### 2.0 Exploratory Data Analysis


In [ ]:
training_df.to_csv("training_df.csv")
training_df

,id,type_code,class_id,inception_date,branch,include_client_consolidation,use_client_address,credit_limit_type,is_arp_locked,dividend_confirm_code,...,is_family_resp,is_hrdc_resp,resp_specimen_plan,inserted_at,updated_at,is_olob,retail_last_maintenance_time,retail_last_maintenance_user,visible_in_reports,label
0,893724,1,3.0,0.808752,5,1,0,0.0,0,0.0,...,0,0,NaN,0.00000,0.835465,0,0.000000,432,0,1
1,1268094,15,5.0,0.847716,5,1,1,0.0,0,0.0,...,0,0,NaN,0.00000,0.860734,0,0.954733,25,1,1
2,606613,18,5.0,0.542538,5,0,0,0.0,0,0.0,...,0,0,NaN,0.00000,0.860734,0,0.633734,174,0,1
3,741930,0,3.0,0.916405,5,1,1,0.0,0,0.0,...,0,0,NaN,0.00000,0.835465,0,0.000000,432,1,1
4,1137922,0,3.0,0.806311,5,1,0,0.0,0,0.0,...,0,0,NaN,0.00000,0.836450,0,0.000000,432,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673334,6827067,8,5.0,0.995206,2,1,1,0.0,1,0.0,...,0,0,NaN,0.34237,0.861687,0,0.989372,25,1,1
673335,590659,13,5.0,0.466963,5,1,0,0.0,0,0.0,...,0,0,NaN,0.00000,0.860734,0,0.626845,25,0,0
673336,1247774,0,3.0,0.873431,5,1,1,0.0,0,0.0,...,0,0,NaN,0.00000,0.835465,0,0.000000,432,1,1
673337,1155640,15,5.0,0.806311,5,1,1,0.0,0,0.0,...,0,0,NaN,0.00000,0.860734,0,0.972643,374,1,1


In [ ]:
# Example of type_code being blank
print(training_df[training_df['id'] == 1955747])
# Example of class_id, inception_date, branch, credit_limit_type, dividend_confirm_code being blank
print(training_df[training_df['id'] == 6778425])


           id  type_code  class_id  inception_date  branch  \
1980  1955747         20       3.0         0.68105       0   

      include_client_consolidation  use_client_address  credit_limit_type  \
1980                             1                   1                0.0   

      is_arp_locked  dividend_confirm_code  ...  is_family_resp  is_hrdc_resp  \
1980              0                    0.0  ...               0             0   

      resp_specimen_plan  inserted_at  updated_at  is_olob  \
1980                 NaN          0.0    0.835465        0   

      retail_last_maintenance_time  retail_last_maintenance_user  \
1980                           0.0                           432   

     visible_in_reports  label  
1980                  0      1  

[1 rows x 64 columns]
         id  type_code  class_id  inception_date  branch  \
85  6778425          9       NaN             0.0       9   

    include_client_consolidation  use_client_address  credit_limit_type  \
85        

Unprocessed columns



Matthew
- special_fee_code: numbers, characters, blanks
- resp_specimen plan: numbers (like ids) and blanks

In [ ]:
# # null_cnt = {}
# # for col in training_df.columns():
# #   null_cnt[col] = training_df[col].isnull().sum()

# # print(null_cnt)

# # Temporary exclusion for training
# training_test_df = training_df.drop(['options_trading_type', 'interest_dividend_conversion_type',
# 'guarantee_gtor_type', 'terminal_code', 'shareholder_language', 'special_tag', 'conjunction', 'title', 'tms_settlement_location', 'portfolio_name_address_option', 'portfolio_report_option',
# 'portfolio_summary_option', 'loan_limit_override', 'non_plan_book_value_flag', 'mailing_consent', 'risk_tolerance', 'special_fee_code', 'non_calendar_year_end', 'resp_specimen_plan'], axis=1)

# X = training_test_df.drop(['id', 'label'], axis = 1)
# # Temporary fill of a missing val within the "label" column
# y = training_test_df.label.fillna(0)

# # Setting random state number to have reproducible results
# seed = 123

# # Splitting the data into the training and testing set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

# # All the models that we want to evaluate and compare through cross-validation
# # models = [('LR', LogisticRegression(random_state=seed)),
# #           ('KNN', KNeighborsClassifier()),
# #           ('CART', DecisionTreeClassifier(random_state=seed)),
# #           ('RF', RandomForestClassifier(random_state=seed)),
# #           ('SVR', SVC(gamma='auto',random_state=seed)),
# #           ('GB', GradientBoostingClassifier(random_state=seed)),
# #           ("LightGBM", LGBMClassifier(random_state=seed)),
# #           ("XtremeGBRFM", XGBRFClassifier(random_state=seed))]

# models = [('KNN', KNeighborsClassifier()),
#           ('CART', DecisionTreeClassifier(random_state=seed)),
#           ('RF', RandomForestClassifier(random_state=seed)),
#           ('GB', GradientBoostingClassifier(random_state=seed)),
#           ("LightGBM", LGBMClassifier(random_state=seed)),
#           ("XtremeGBRFM", XGBRFClassifier(random_state=seed))]

# # Results array to store the cross validation scores (can also change to evaluate for accuracy)
# results = []
# # Names array to store the name of each model
# names = []

# # Store max cv score to automatically choose highest cv score model to use
# # max_cv_score = 0
# # max_cv_model = ""

# # Loop through the array of models evaluating cross-validation scores for each model
# for name, model in models:
#   # Setup the KFold cross-validation object
#   kfold = KFold(n_splits=10, random_state=seed, shuffle=True)
#   # Get the cross-validation scores for each model
#   cv_results = cross_val_score(model, X, y, cv=kfold)
#   if (cv_results > max_cv_score):
#     max_cv_score = cv_results
#     # max_cv_model = name
#   # Add the results and names to their respective arrays
#   results.append(cv_results)
#   names.append(name)
#   msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
#   print(msg)
# # Evaluate each models results (First number is avg cross-validation score. Second number is std dev for cross-validation scores)
# # Choose best model from here



KNN: 0.962021 (0.000715)
CART: 0.948063 (0.000548)
RF: 0.963207 (0.000633)
GB: 0.968248 (0.000621)
[LightGBM] [Info] Number of positive: 484383, number of negative: 121622
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.234961 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2660
[LightGBM] [Info] Number of data points in the train set: 606005, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.799305 -> initscore=1.381958
[LightGBM] [Info] Start training from score 1.381958
[LightGBM] [Info] Number of positive: 484565, number of negative: 121440
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.121362 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2674
[LightGBM] [Info] Number of data points in the train set: 606005, number 

In [ ]:
test_set = pd.read_csv('test.csv')


<ipython-input-14-01fbb2699312>:1: DtypeWarning: Columns (62,64,87) have mixed types. Specify dtype option on import or set low_memory=False.
  test_set = pd.read_csv('test.csv')


In [ ]:
# model =

# model.fit(X, y)

# prediction_df = test_set
# predictions = []

# # make predictions using decided model
# from numpy import asarray

# for row, column in training_test_df.iterrows():
#   predictions.append(model.predict(row))

# prediction_df['label'] = predictions

